The dataset consists of the printing parameters and the quality evaluation of the printed cubes.


In [ ]:
import pandas as pd
import numpy as np

printing_parameters = [
    "Pump_pressure",
    "Print_speed",
    "Infill_density",
    "Nozzle_gauge"
]

evaluation_parameters = [
    "No_of_cracks",
    "Density",
    "Roughness"
    "Date_evaluated"
]

data = pd.DataFrame(columns=printing_parameters + evaluation_parameters)

Defining X_train to be the printing parameters and Y_train to be the evaluation parameters. 

Standardizing the data for better performance of the Gaussian Process Regressor.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = data[printing_parameters].values
y_train = data[evaluation_parameters].values

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
y_train_scaled = y_scaler().fit_transform(y_train)

Fitting the GPR model to the standardized data.

In [ ]:
# GPR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF,
    Matern,
    RationalQuadratic,
    WhiteKernel,
    ConstantKernel,
)

kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(
    length_scale=1.0, length_scale_bounds=(1e-2, 1e2)
)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
gp.fit(X_train, y_train)

# Dense grid for predictions within the input parameter space. Create
# linspaces for each printing parameter in the input space defined by the
# min and max values supported by the equipment.
input_space_lims = {
    "Pump_pressure": (20, 700),
    "Print_speed": (1, 5),
    "Infill_density": (0, 100),
    "Nozzle_gauge": (18, 30),
}

# Easier visualization by keeping two parameters constant and varying the other two.
pump_pressure_fixed = 20
Nozzle_gauge_fixed = 18

print_speed_lin = np.linspace(1, 5, 50)
infill_density_lin = np.linspace(0, 100, 50)

X1, X2 = np.meshgrid(print_speed_lin, infill_density_lin)

# Prepare the grid for prediction
X_pred = np.zeros((X1.size, len(printing_parameters)))
X_pred[:, 0] = pump_pressure_fixed  # Pump_pressure
X_pred[:, 1] = X1.ravel()           # Print_speed
X_pred[:, 2] = X2.ravel()           # Infill_density
X_pred[:, 3] = Nozzle_gauge_fixed   # Nozzle_gauge

# Scale the prediction inputs
X_pred_scaled = X_scaler.transform(X_pred)
# Make predictions
y_pred_scaled, y_std_scaled = gp.predict(X_pred_scaled, return_std=True)
# Inverse scale the predictions
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_std = y_std_scaled * y_scaler.scale__  # Adjust std deviation accordingly

# Reshape predictions back to grid shape for visualization
y_pred_grid = y_pred.reshape(X1.shape + (len(evaluation_parameters),))
y_std_grid = y_std.reshape(X1.shape + (len(evaluation_parameters),))

# Now y_pred_grid and y_std_grid can be used for visualization or further analysis

# Plotting Cracks vs Print_speed and Infill_density
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.contourf(X1, X2, y_pred_grid[:, :, 0], levels=20, cmap='viridis')
plt.colorbar(label='Predicted Number of Cracks')
plt.xlabel('Print Speed')
plt.ylabel('Infill Density')
plt.title('Predicted Number of Cracks vs Print Speed and Infill Density')
plt.show()
